In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import time
import os
import sys
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))

t1 = time.time()
# os.environ['THEANO_FLAGS'] = "device=cuda0"

In [ ]:
import shelve
from pprint import pprint
import numpy as np
import lasagne
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
import matplotlib.cm
from matplotlib import gridspec
from lproc import subset, rmap
from sltools.utils import gloss2seq, seq2gloss
from sltools.nn_utils import onehot, jaccard, compute_scores
from sltools.postproc import optimize_boundaries, filter_longshort

from experiments.hmmvsrnn_reco.a_data import durations, gloss_seqs, tmpdir, \
   train_subset, val_subset, test_subset, vocabulary
from experiments.hmmvsrnn_reco.utils import reload_best_hmm

In [ ]:
if 'EXPERIMENT_NAME' in os.environ:
    experiment_name = os.environ['EXPERIMENT_NAME']
else:
    experiment_name = "hmm_skel_tc15"

report = shelve.open(os.path.join(tmpdir, experiment_name))

model = report['meta']['model']
modality = report['meta']['modality']
variant = report['meta']['variant']
date = report['meta']['date']
notes = report['meta']['notes']
experiment_name = report['meta']['experiment_name']
args = report['args']
encoder_kwargs = args['encoder_kwargs']

# Reload dataset

In [ ]:
if modality == "skel":
    from experiments.hmmvsrnn_reco.b_preprocess import skel_feat_seqs
    feat_seqs = [skel_feat_seqs]
elif modality == "bgr":
    from experiments.hmmvsrnn_reco.b_preprocess import bgr_feat_seqs
    feat_seqs = [bgr_feat_seqs]
elif modality == "fusion":
    from experiments.hmmvsrnn_reco.b_preprocess import skel_feat_seqs
    from experiments.hmmvsrnn_reco.b_preprocess import bgr_feat_seqs
    feat_seqs = [skel_feat_seqs, bgr_feat_seqs]
elif modality == "transfer":
    from experiments.hmmvsrnn_reco.b_preprocess import transfer_feat_seqs
    feat_seqs = transfer_feat_seqs(encoder_kwargs['transfer_from'],
                                   encoder_kwargs['freeze_at'])
else:
    raise ValueError()

feat_seqs_train = [subset(f, train_subset) for f in feat_seqs]
gloss_seqs_train = subset(gloss_seqs, train_subset)
durations_train = subset(durations, train_subset)
targets_train = rmap(lambda g, d: gloss2seq(g, d, 0),
                     gloss_seqs_train, durations_train)
feat_seqs_val = [subset(f, val_subset) for f in feat_seqs]
gloss_seqs_val = subset(gloss_seqs, val_subset)
durations_val = subset(durations, val_subset)
targets_val = rmap(lambda g, d: gloss2seq(g, d, 0),
                    gloss_seqs_val, durations_val)
feat_seqs_test = [subset(f, test_subset) for f in feat_seqs]
gloss_seqs_test = subset(gloss_seqs, test_subset)
durations_test = subset(durations, test_subset)
targets_test = rmap(lambda g, d: gloss2seq(g, d, 0),
                    gloss_seqs_test, durations_test)

# Training report and model

In [ ]:
plt.figure(figsize=(8, 4))

i = 0
for e in sorted([e for e in report.keys() if e.startswith('epoch')]):
    r = report[e]    
    plt.plot(np.arange(i, i + len(r['epoch_losses'])), r['epoch_losses'], c='blue')
    plt.scatter([i + len(r['epoch_losses']) - 1], [r['epoch_losses'][-1]], 
                marker='x', c='red', alpha=.5)
    
    i += len(r['epoch_losses'])

plt.gca().set_yscale("log")
plt.show()

best_epoch, recognizer, previous_recognizer = reload_best_hmm(report)
print("best epoch:", best_epoch)

# Performances

In [ ]:
def plot_perfs(perf_report, chains_lengths):
    fig = plt.figure(figsize=(10, 5), dpi=150)
    gs = gridspec.GridSpec(2, 3)
    
    ax = fig.add_subplot(gs[0, 0])
    cmap = matplotlib.cm.viridis
    cmap.set_bad(cmap(0.001))
    plt.imshow(perf_report['confusion'] / perf_report['confusion'].sum(axis=1, keepdims=True), 
               interpolation='none', 
               cmap=cmap, clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title("HMM Jaccard/Framewise:\n{:5.4f}/{:5.4f}".format(
        perf_report['jaccard'], perf_report['framewise']))

    ax = fig.add_subplot(gs[1, 0])
    cmap = matplotlib.cm.viridis
    cmap.set_bad(cmap(0.001))
    ax.imshow(perf_report['posterior_confusion'] / perf_report['posterior_confusion'].sum(axis=1, keepdims=True), 
              interpolation='none', 
              cmap=cmap, clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title("Posterior Jaccard/Framewise:\n{:5.4f}/{:5.4f}".format(
        perf_report['posterior_jaccard'], perf_report['posterior_framewise']))

    ax = fig.add_subplot(gs[0:, 1:])
    cmap = matplotlib.cm.viridis
    cmap.set_bad(cmap(0.001))
    ax.imshow(perf_report['statewise_confusion'] / perf_report['statewise_confusion'].sum(axis=1, keepdims=True), 
              interpolation='none', 
              cmap=cmap, clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks(np.cumsum(chains_lengths) - .5)
    ax.set_yticks(np.cumsum(chains_lengths) - .5)
    ax.grid(color='gray', linestyle='dotted')
    ax.set_title("State-wise framewise: {:5.4f}".format(perf_report['statewise_framewise']))
    
    plt.colorbar(ax=ax)
    
    fig.tight_layout(w_pad=-20)

In [ ]:
plot_perfs(report[best_epoch]['train_scores'], recognizer.chains_lengths)
plot_perfs(report[best_epoch]['val_scores'], recognizer.chains_lengths)

In [ ]:
perf_report = report[best_epoch]['val_scores']
chains_lengths = recognizer.chains_lengths

fig = plt.figure(figsize=(7, 6), dpi=150)
ax = fig.add_subplot(111)
cmap = matplotlib.cm.viridis
cmap.set_bad(cmap(0.001))
im = ax.imshow(perf_report['statewise_confusion'] / perf_report['statewise_confusion'].sum(axis=1, keepdims=True), 
          interpolation='none', 
          cmap=cmap, clim=(0.001, 1), norm=LogNorm(vmin=0.001, vmax=1))
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xticks(np.cumsum(chains_lengths) - .5)
ax.set_yticks(np.cumsum(chains_lengths) - .5)
ax.spines['left'].set_position(('outward', 2))
ax.spines['bottom'].set_position(('outward', 2))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(color='w', linestyle='--', linewidth=.5)
ax.set_title("accuracy: {:4.3f}".format(perf_report['statewise_framewise']))

fig.colorbar(im)

fig.savefig("/home/granger/exp1_state_confusion.pdf", bbox_inches='tight')

# Error

In [ ]:
targets_train = [gloss2seq(g_, d_, 0) for g_, d_ in zip(gloss_seqs_train, durations_train)]
predictions_train = recognizer.predict(feat_seqs_train)
ji_train, framewise_train, confusion_train = compute_scores(predictions_train, targets_train, vocabulary)

targets_val = [gloss2seq(g_, d_, 0) for g_, d_ in zip(gloss_seqs_val, durations_val)]
predictions_val = recognizer.predict(feat_seqs_val)
ji_val, framewise_val, confusion_val = compute_scores(predictions_val, targets_val, vocabulary)

In [ ]:
# distribution of errors

scores = np.array([jaccard(onehot(l, vocabulary), onehot(p, vocabulary))
                   for l, p in zip(targets_val, predictions_val)])

plt.figure(figsize=(4, 2), dpi=150)
h, bins = np.histogram(scores, np.linspace(0.0, 1, 41))
h = h / np.sum(h) * 40
plt.bar(bins[:-1], h, width=1/40, 
        align='center', color="lightslategray", edgecolor="darkslategray")
plt.xlabel("Sequence Jaccard Index")
plt.ylabel("density")

plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, linestyle='--', linewidth=.5)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.savefig("/home/granger/exp1_ji_density.pdf", bbox_inches='tight')

In [ ]:
# nb of false positives out of sequence vocabulary
np.mean([len(set(p_) - set(l_)) for p_, l_ in zip(predictions_val, targets_val)], axis=0)

In [ ]:
# confusion types
cum_err = np.sum(confusion_val, axis=1) - np.diag(confusion_val)
print("false pos: {}  false neg: {}, mis-class: {}".format(
    cum_err[0], np.sum(confusion_val[1:, 0]), np.sum(cum_err[1:]) - np.sum(confusion_val[1:, 0])))

In [ ]:
# Plot the posteriors of the _correct_ states in color and other states in gray

def preview_seq(proba, gloss):
    plt.figure(figsize=(10, 1), dpi=150)
    cmap = plt.cm.summer(np.linspace(0, 1, 5))
    
    pre_start = 0
    for lbl, start, stop in gloss:
        for i in range(proba.shape[1] - 1):
            plt.plot(np.arange(pre_start, start), proba[pre_start:start, i], ls=':', c="gray")
        plt.plot(np.arange(pre_start, start), proba[pre_start:start, -1], c="purple")
        for a in range(0, (lbl - 1) * 5):
            plt.plot(np.arange(start, stop), proba[start:stop, a], ls=":", c='gray')
        for a in range(5):
            plt.plot(np.arange(start, stop), proba[start:stop, (lbl - 1) * 5 + a], c=cmap[a])
        for a in range(lbl * 5, proba.shape[1]):
            plt.plot(np.arange(start, stop), proba[start:stop, a], ls=":", c='gray')
        pre_start = stop
    
    for i in range(proba.shape[1] - 1):
        plt.plot(np.arange(pre_start, len(proba)), proba[pre_start:len(proba), i], ls=':', c="gray")
    plt.plot(np.arange(pre_start, len(proba)), proba[pre_start:len(proba), -1], c="purple")
    
    plt.gca().set_axisbelow(True)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    
seq = 15

preview_seq(
    recognizer.posterior.predict_proba(*[f[seq] for f in feat_seqs_val]),
    gloss_seqs_val[seq])

plt.gca().set_xlim((200, 900))

In [ ]:
recognizer.hmm.__getstate__().keys()

In [ ]:
edges = recognizer.hmm.__getstate__()['edges']
states = recognizer.hmm.__getstate__()['states']

out_edges = {int(states[v1].name): [] for v1, *_ in edges}
for v1, v2, w, _, _ in recognizer.hmm.__getstate__()['edges']:
    # print("{:>3d} -> {:>3d}: {:.5f}".format(v1, v2, w))
    out_edges[int(states[v1].name)].append((int(states[v2].name), w))

In [ ]:
fig = plt.figure(figsize=(7, 5), dpi=150)
gs = gridspec.GridSpec(4, 7, width_ratios=[1, 1, 1, 1, 1, .0, .1])
limit = 1e-5

cnames = [
    'vattene','vieniqui','perfetto','furbo','cheduepalle','chevuoi','daccordo',
    'seipazzo','combinato','freganiente','ok','cosatifarei','basta','prendere',
    'noncenepiu','fame','tantotempo','buonissimo','messidaccordo','sonostufo']

for c in range(20):
    ax = fig.add_subplot(gs[c // 5, c % 5])
    state_transition_matrix = np.zeros((5, 6))
    
    for i, s1 in enumerate(range(c * 5, c * 5 + 5)):
        for s2, w in out_edges[s1]:
            if s2 == 100:
                state_transition_matrix[i, -1] = w
            else:
                state_transition_matrix[i, s2 - c * 5] = w
    
    state_transition_matrix = np.clip(state_transition_matrix, limit, 1)
    
    im = ax.imshow(state_transition_matrix, clim=(limit, 1), norm=LogNorm(limit, vmax=1))
    ax.axis('off')
    ax.set_title(cnames[c])

bar_ax = fig.add_subplot(gs[1:3, 6])
fig.colorbar(im, ax=bar_ax, cax=bar_ax)
fig.tight_layout(pad=0, h_pad=-10)

fig.savefig("/home/granger/exp1_transition_matrices.pdf", bbox_inches='tight')

In [ ]:
# correlate error with predicted gloss duration

plt.figure(figsize=(9, 4))

prediction_accuracy = [np.sum(l[start:stop] == g) 
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
none_accuracy = [np.sum(l[start:stop] == 0)
            for p, l in zip(predictions_val, targets_val)
            for (g, start, stop) in seq2gloss(p)
            if g != 0]
gloss_d = [stop - start
           for p in predictions_val 
           for (g, start, stop) in seq2gloss(p)
           if g != 0]

scores_pred = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
scores_none = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
total_d = np.zeros((int(np.ceil(max(gloss_d) / 5 + 0.0001)),))
for vp, vn, d in zip(prediction_accuracy, none_accuracy, gloss_d):
    idx = int(d / 5)
    scores_pred[idx] += vp
    scores_none[idx] += vn
    total_d[idx] += d

plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5), 
              scores_pred / total_d,
              width=5,
              alpha=.5)
plt.gca().bar(np.arange(0, int(np.ceil(max(gloss_d) + 0.0001)), 5), 
              scores_none / total_d,
              width=5,
              alpha=.5)

plt.legend(["predicted class", "non-gesture class"])
plt.xlabel("subsequence duration (based on model prediction)")
plt.ylabel("accuracy")
plt.show()

# Score

In [ ]:
# Optimize duration filter

boundaries = optimize_boundaries(targets_val, predictions_val, vocabulary, (30, 100, 301))
print("Optimal range: ", boundaries)
print("FYI the score without is: {:.4f}".format(ji_val))

In [ ]:
# Validation score

ji_filtered_val, accuracy_filtered_val, confusion_filtered_val = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_val], 
    targets_val, vocabulary)
print("validation score: {:.4f}".format(ji_filtered_val))

In [ ]:
# Test score

targets_test = [gloss2seq(g_, d_, 0) for g_, d_ in zip(gloss_seqs_test, durations_test)]
predictions_test = recognizer.predict(feat_seqs_test)

ji_filtered_test, accuracy_filtered_test, confusion_filtered_test = compute_scores(
    [filter_longshort(p, boundaries, 0) for p in predictions_test], 
    targets_test, vocabulary)
print("testing score: {:.4f}".format(ji_filtered_test))

In [ ]:
# Recap:

print("Accuracy:   {:.4f} / {:.4f} / ?".format(framewise_train, framewise_val))
print("JI:         {:.4f} / {:.4f} / ?".format(ji_train, ji_val))
print("Acc. filt.:      ? / {:.4f} / {:.4f}".format(accuracy_filtered_val, accuracy_filtered_test))
print("JI filt.:        ? / {:.4f} / {:.4f}".format(ji_filtered_val, ji_filtered_test))

recap = {
    "experiment_name": experiment_name,
    "best_epoch": best_epoch,
    "accuracy": (framewise_train, framewise_val, None),
    "ji": (ji_train, ji_val, None),
    "confusion": (confusion_train, confusion_val, None),
    "accuracy_filtered": (None, accuracy_filtered_val, accuracy_filtered_test),
    "ji_filtered": (None, ji_filtered_val, ji_filtered_test),
    "confusion_filtered": (None, confusion_filtered_val, None),
}
report['analysis'] = recap

# Model

In [ ]:
print(report['args']['encoder_kwargs'])

In [ ]:
# from sklearn.manifold import TSNE
# from sltools.tconv import TemporalConv

# l = None
# for l_ in lasagne.layers.get_all_layers(recognizer.posterior.l_feats):
#     if isinstance(l_, TemporalConv):
#         l = l_

# W1 = l.W.eval()
# W1 = W1.transpose((0, 2, 1)).reshape((-1, W1.shape[1]))
# Y = np.linalg.norm(W1, axis=1)
# i = np.argsort(Y)
# W1 = np.stack([W1[i_] for i_ in i])[-300:]
# model = TSNE(n_components=1, metric='euclidean')
# Y = model.fit_transform(W1)[:, 0]
# i = np.argsort(Y)
# W1 = W1[i]
# Y = Y[i]

# plt.figure(figsize=(5, 20))
# x, y = np.meshgrid(np.arange(W1.shape[1]), Y)
# plt.imshow(W1, clim=(-np.abs(W1).max(), np.abs(W1).max()), cmap='bwr')
# plt.gca().set_aspect("auto")
# plt.axis([-1, W1.shape[1], -1, W1.shape[0]])

In [ ]:
# t2 = time.time()
# print(t2 - t1)

In [ ]:
# l = lasagne.layers.get_all_layers(recognizer.posterior.l_feats)[2]
# W = np.asarray(l.W.eval())

# nrows, ncols = int(np.ceil(np.sqrt(W.shape[0] + 1))), int(np.floor(np.sqrt(W.shape[0] + 1)))
# img = np.zeros((nrows * (W.shape[2] + 1), ncols * (W.shape[3] + 1)))
# for k in range(W.shape[0]):
#     i, j = k // ncols, k % ncols
#     y, x = i * (W.shape[2] + 1), j * (W.shape[3] + 1)
#     tmp = img[y:y + W.shape[2]]
#     img[y:y + W.shape[2], x:x + W.shape[3]] = W[k, 0]

# plt.imshow(img, clim=(-np.abs(W).max(), np.abs(W).max()), cmap='bwr')
# plt.colorbar()

In [ ]:
# lbl, cnt = np.unique(
#     np.concatenate([gloss2seq(g_, len(r_), 0) for g_, r_ in zip(gloss_seqs_val, feat_seqs_val)]),
#     return_counts=True)

In [ ]:
# plt.bar(x=lbl+.5, height=cnt, log=True)
# plt.gca().set_ylim((1, 3e5))

In [ ]:
# plt.bar(x=lbl+.5, height=cnt, log=False)
# plt.gca().set_ylim((1, 2e5))
# plt.show()

In [ ]:
# sum([len(gseq) for gseq in gloss_seqs_val])